In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [2]:
import pymysql
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
plt.rc('font', family = 'Hancom Gothic')

In [8]:
import joblib
model = joblib.load('M2_categorical_LGBM.pkl')
pm_gu_list =joblib.load('M2_ENCODER\PM활용행정구리스트.pkl')
gu_list = joblib.load('M2_ENCODER\서울전역행정구리스트.pkl')
le_season = joblib.load('M2_ENCODER\계절라벨인코더.pkl')
le_commute =joblib.load('M2_ENCODER\출퇴근시간여부라벨인코더.pkl')
le_gu = joblib.load('M2_ENCODER\행정구라벨인코더.pkl')
cols_scale = joblib.load('M2_ENCODER\독립변수리스트.pkl')
scaler = joblib.load('M2_ENCODER\독립변수스케일러.pkl')
le_target = joblib.load('M2_ENCODER\타겟변수라벨인코더.pkl')

In [7]:
def class_to_range(pred_class):
    ''' 모델 예측 결과를 범주형 결과로 반환하는 함수 '''
    labels = ['공급과다', '공급평균', '공급다소부족', '공급부족', '공급절대부족']
    pred_class = int(pred_class)
    return labels[pred_class]

In [72]:
import pymysql
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta

def DBMS_last_year_one_week() :
    '''내일을 기준으로 작년 1주일치 생활인구수와 대여량을 DataFrame으로 반환하는 함수'''
    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]

    for data in dt_1y :
        query =f"""
            SELECT *
            FROM basic_data
            WHERE year = {data.year}
              AND month = {data.month}
              AND day = {data.day}
              """

        try:
            connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database,
                port=port,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            with connection.cursor() as cursor:
                cursor.execute(query)
                data = cursor.fetchall()
                whole_data = pd.DataFrame(data)
                whole_data.columns = ['연도','월','일','시','행정구','일시','1년전_총생활인구수','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','1년전_공공자전거대여량']
        except Exception as e:
            print('에러 발생:', e)
        finally:
            try:
                connection.close()
            except:
                pass
    return whole_data # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전

In [71]:
DBMS_last_year_one_week()

,연도,월,일,시,행정구,일시,1년전_총생활인구수,계절,출퇴근시간여부,주말구분,강수,기온,습도,풍속,공공자전거대여량
0,2024,7,16,0,강남구,2024-07-16 00:00:00,695981,여름,기타,0,0.0,29.1,62.0,1.9,122
1,2024,7,16,0,강동구,2024-07-16 00:00:00,534823,여름,기타,0,0.0,29.6,63.0,2.1,116
2,2024,7,16,0,강북구,2024-07-16 00:00:00,284486,여름,기타,0,0.0,29.8,62.0,1.0,58
3,2024,7,16,0,강서구,2024-07-16 00:00:00,545402,여름,기타,0,0.0,29.5,61.0,2.3,230
4,2024,7,16,0,관악구,2024-07-16 00:00:00,493880,여름,기타,0,0.0,30.5,57.0,2.1,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2024,7,16,23,용산구,2024-07-16 23:00:00,237520,여름,기타,0,26.0,23.6,94.0,2.3,6
596,2024,7,16,23,은평구,2024-07-16 23:00:00,463000,여름,기타,0,23.0,24.5,97.0,0.6,18
597,2024,7,16,23,종로구,2024-07-16 23:00:00,202044,여름,기타,0,20.3,25.2,97.0,2.6,13
598,2024,7,16,23,중구,2024-07-16 23:00:00,172396,여름,기타,0,20.3,25.2,97.0,2.6,8


In [65]:
import requests
import pandas as pd

def get_hourly_weather_seoul_openmeteo():
    SEOUL_DISTRICTS = [
        {"name": "강남구", "lat": 37.5172, "lon": 127.0473},
        {"name": "강동구", "lat": 37.5301, "lon": 127.1238},
        {"name": "강북구", "lat": 37.6396, "lon": 127.0256},
        {"name": "강서구", "lat": 37.5509, "lon": 126.8495},
        {"name": "관악구", "lat": 37.4784, "lon": 126.9516},
        {"name": "광진구", "lat": 37.5384, "lon": 127.0823},
        {"name": "구로구", "lat": 37.4955, "lon": 126.8878},
        {"name": "금천구", "lat": 37.4604, "lon": 126.9006},
        {"name": "노원구", "lat": 37.6542, "lon": 127.0568},
        {"name": "도봉구", "lat": 37.6688, "lon": 127.0472},
        {"name": "동대문구", "lat": 37.5744, "lon": 127.0396},
        {"name": "동작구", "lat": 37.5124, "lon": 126.9392},
        {"name": "마포구", "lat": 37.5663, "lon": 126.9014},
        {"name": "서대문구", "lat": 37.5791, "lon": 126.9368},
        {"name": "서초구", "lat": 37.4836, "lon": 127.0327},
        {"name": "성동구", "lat": 37.5633, "lon": 127.0364},
        {"name": "성북구", "lat": 37.5894, "lon": 127.0167},
        {"name": "송파구", "lat": 37.5146, "lon": 127.1056},
        {"name": "양천구", "lat": 37.5179, "lon": 126.8666},
        {"name": "영등포구", "lat": 37.5264, "lon": 126.8962},
        {"name": "용산구", "lat": 37.5323, "lon": 126.9909},
        {"name": "은평구", "lat": 37.6027, "lon": 126.9291},
        {"name": "종로구", "lat": 37.5735, "lon": 126.9790},
        {"name": "중구", "lat": 37.5636, "lon": 126.9977},
        {"name": "중랑구", "lat": 37.6065, "lon": 127.0927}
    ]

    result_rows = []
    for gu in SEOUL_DISTRICTS:
        latitude, longitude, name = gu["lat"], gu["lon"], gu["name"]
        url = (
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            "&hourly=temperature_2m,precipitation,weathercode,relative_humidity_2m,wind_speed_10m"
            "&forecast_days=8&timezone=Asia%2FSeoul"
        )
        resp = requests.get(url)
        if resp.status_code == 200:
            json_data = resp.json()
            if 'hourly' not in json_data:
                print(f"[경고] {name} 데이터에 'hourly' 없음. 응답: {json_data}")
                continue
            data = json_data['hourly']
            for t, temp, rain, code, rh, wind in zip(
                data['time'],
                data['temperature_2m'],
                data['precipitation'],
                data['weathercode'],
                data['relative_humidity_2m'],
                data['wind_speed_10m'],
            ):
                result_rows.append({
                    "행정구": name,
                    "일시": t,
                    "강수": rain,
                    "습도": rh,
                    "풍속": wind,
                    "기온": temp,
                })
        else:
            print(f"[오류] {name} API 호출 실패: {resp.status_code}")

    # DataFrame 생성
    df = pd.DataFrame(result_rows)

    # 일시 → datetime 변환 및 파생 변수 추출
    df['일시'] = pd.to_datetime(df['일시'])
    df['연'] = df['일시'].dt.year
    df['월'] = df['일시'].dt.month
    df['일'] = df['일시'].dt.day
    df['시'] = df['일시'].dt.hour
    df['날짜'] = df['일시'].dt.date  # 날짜만 필요할 때

    return df

In [66]:
cols_X = ['연도','월','일','시','행정구','총생활인구수','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','공공자전거대여량','공휴일','1년전_총생활인구수','1년전_PM대여량']

In [67]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import holidays
import tensorflow as tf

# 예측 함수
def predict_bike_demand():
    """
    내일부터 7일간의 자전거 수요를 예측하는 함수
    """
    le = joblib.load('ENCODER_SCALER/labelencoders.pkl')
    scaler = joblib.load('ENCODER_SCALER/scaler.pkl')
    feature_cols = joblib.load('ENCODER_SCALER/featurecols.pkl')
    num_cols = joblib.load('ENCODER_SCALER/numcols.pkl')
    model = tf.keras.models.load_model('best_dnn_model_lr.h5')

    df_1y = DBMS_last_year_one_week()[['월', '일', '시', '행정구', '총생활인구수_1년전', '대여량_1년전']] # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전
    weather_df = get_hourly_weather_seoul_openmeteo()[['행정구', '월', '일', '시','강수', '습도', '풍속', '기온','일시']]
    weather_df['일시'] = pd.to_datetime(weather_df['일시'])
    df_total = weather_df.merge(df_1y, on=['행정구','월','일','시'])
    
    kr_holidays = holidays.KR(years=[datetime.now().year, datetime.now().year+1])
    df_total['요일'] = df_total['일시'].dt.weekday
    df_total['주말구분'] = (df_total['요일'] >= 5).astype(int)
    df_total['공휴일'] = df_total['일시'].dt.date.isin(kr_holidays).astype(int)

#     input_df = pd.DataFrame([input_dict])
    df_total['행정구'] = le.transform(df_total['행정구'].values)
    df_total[num_cols] = scaler.transform(df_total[num_cols])
    X_input = df_total[feature_cols].astype('float32').values
    pred = model.predict(X_input,verbose=0).round().astype(np.int32)
    return pred

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
def result_df_return() :
    '''위 함수를 토대로 8일 24시간 동안의 예상 대여량을 dataframe으로 반환하는 함수'''
    datas = predict_bike_demand()
    districts = ['강남구',"강동구","강북구", "강서구","관악구","광진구","구로구","금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"]
    
    base_date = datetime.now()
    num_days = 8
    num_hours = 24
    all_dates = [base_date + timedelta(days=i) for i in range(num_days)]

    date_hour_list = []
    for d in all_dates:
        for h in range(num_hours):
            date_hour_list.append((d.date(), h))

    # datas shape 자동 보정
    datas = np.array(datas).reshape(len(districts), len(date_hour_list))  
    
    df = pd.DataFrame(datas, index=districts, columns=pd.MultiIndex.from_tuples(date_hour_list, names=['날짜', '시']))

    df_long = df.stack(level=[0, 1]).reset_index()
    df_long.columns = ['행정구', '날짜', '시', '예측값']

#     print(df_long.head())
#     print(df_long.shape)  # (3600, 4) => 현재는 7일치로, 4200,4
    return df_long

In [84]:
import holidays
def predict_result() :
    '''오늘로부터 8일간의 데이터를 predict해서 결과를 반환하는 함수'''
    df_1y = DBMS_last_year_one_week() # 연도	월	일	시	행정구	일시	1년전_총생활인구수	계절	출퇴근시간여부	주말구분	강수	기온	습도	풍속	공공자전거대여량
    pm_gu_list =joblib.load('M2_ENCODER\PM활용행정구리스트.pkl')
    PM_data = pd.read_csv('PM_data/서울시_시간별_PM_배치_2021_2025.csv')
    PM_data.columns = ['일시', '행정구', '1년전_PM대여량']
    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]
    target_dates = [d.date() for d in dt_1y]
    PM_data['일시'] = pd.to_datetime(PM_data['일시'])

    PM_data['연도'] = PM_data['일시'].dt.year
    PM_data['월'] = PM_data['일시'].dt.month
    PM_data['일'] = PM_data['일시'].dt.day
    PM_data['시'] = PM_data['일시'].dt.hour
    
    mask = PM_data['일시'].dt.date.isin(target_dates)
    filtered = PM_data[mask]
    
    
    df_total = df_1y.merge(filtered, on=['행정구','월','일','시','일시','연도'])
#     df_total['일시'] = pd.to_datetime(df_total['일시'])
    
    feature_cols = ['PM대여량', '1년전_PM대여량', '1년전_총생활인구수', '총생활인구수', '출퇴근시간여부', '시', '주말구분',
       '풍속', '공공자전거대여량', '습도', '기온', '행정구', '월', 'PM대여량_class', '계절', '강수',
       '연도', '공휴일', '일']
    
    kr_holidays = holidays.KR(years=[datetime.now().year, datetime.now().year+1])
#     df_total['요일'] = df_total['일시'].dt.weekday
#     df_total['주말구분'] = (df_total['요일'] >= 5).astype(int)
#     df_total['공휴일'] = df_total['일시'].dt.date.isin(kr_holidays).astype(int)
#     df_need = df_total[['연도','월','일','시','행정구','총생활인구수','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','공공자전거대여량','공휴일','1년전_총생활인구수','1년전_PM대여량']]
    
#     weather_df = get_hourly_weather_seoul_openmeteo()[['행정구', '월', '일', '시','강수', '습도', '풍속', '기온','일시']]
#     weather_df['일시'] = pd.to_datetime(weather_df['일시'])

# #     input_df = pd.DataFrame([input_dict])
#     df_total['행정구'] = le.transform(df_total['행정구'].values)
#     df_total[num_cols] = scaler.transform(df_total[num_cols])
#     X_input = df_total[feature_cols].astype('float32').values
#     pred = model.predict(X_input,verbose=0).round().astype(np.int32)
    return df_total

## 데이터 만드는중

In [85]:
filtered

,일시,행정구,1년전_PM대여량
585960,2024-07-09 00:00:00,강남구,33
585961,2024-07-09 01:00:00,강남구,33
585962,2024-07-09 02:00:00,강남구,33
585963,2024-07-09 03:00:00,강남구,33
585964,2024-07-09 04:00:00,강남구,33
...,...,...,...
589603,2024-07-16 19:00:00,은평구,92
589604,2024-07-16 20:00:00,은평구,64
589605,2024-07-16 21:00:00,은평구,64
589606,2024-07-16 22:00:00,은평구,64


In [86]:
predict_result()

,연도,월,일,시,행정구,일시,1년전_총생활인구수,계절,출퇴근시간여부,주말구분,강수,기온,습도,풍속,1년전_공공자전거대여량,1년전_PM대여량
0,2024,7,16,0,강남구,2024-07-16 00:00:00,695981,여름,기타,0,0.0,29.1,62.0,1.9,122,30
1,2024,7,16,0,강동구,2024-07-16 00:00:00,534823,여름,기타,0,0.0,29.6,63.0,2.1,116,27
2,2024,7,16,0,강북구,2024-07-16 00:00:00,284486,여름,기타,0,0.0,29.8,62.0,1.0,58,4
3,2024,7,16,0,강서구,2024-07-16 00:00:00,545402,여름,기타,0,0.0,29.5,61.0,2.3,230,4
4,2024,7,16,0,관악구,2024-07-16 00:00:00,493880,여름,기타,0,0.0,30.5,57.0,2.1,101,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2024,7,16,23,송파구,2024-07-16 23:00:00,728853,여름,기타,0,37.0,23.7,95.0,1.3,41,50
452,2024,7,16,23,양천구,2024-07-16 23:00:00,384987,여름,기타,0,8.0,25.2,94.0,1.6,44,21
453,2024,7,16,23,영등포구,2024-07-16 23:00:00,421582,여름,기타,0,21.5,25.6,100.0,1.4,55,7
454,2024,7,16,23,은평구,2024-07-16 23:00:00,463000,여름,기타,0,23.0,24.5,97.0,0.6,18,6
